In [ ]:
import os

# Set your Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'danish2804'
os.environ['KAGGLE_KEY'] = '787d6e4072b4a739cca74620eca94fc1'








In [ ]:
from flask import Flask, request, jsonify
import os
import pandas as pd
import numpy as np
import torch
import string
import re
from transformers import BertTokenizer, BertForMaskedLM

app = Flask(__name__)


os.environ['KAGGLE_USERNAME'] = 'danish2804'
os.environ['KAGGLE_KEY'] = '787d6e4072b4a739cca74620eca94fc1'


df = pd.read_csv('articles1.csv')

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def preprocess(text):

    text = re.sub(r'\W+', ' ', text)
  
    text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
 
    text = text.translate(str.maketrans('', '', string.punctuation))
 
    if not text:
        return None
    return text
df['content'] = df['content'].apply(preprocess)

df = df.dropna(subset=['content']).reset_index(drop=True)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')


def generate_text(prompt, max_length=400):
    
    tokenized_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')

  
    for i in range(max_length):
   
        predictions = model(tokenized_prompt)[0]
        predicted_token = predictions[0, -1].argmax().item()


        predicted_word = tokenizer.convert_ids_to_tokens(predicted_token)


        prompt = prompt[:512]

        tokenized_prompt = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')
        tokenized_prompt = torch.cat([tokenized_prompt, torch.tensor([[predicted_token]]).to(tokenized_prompt.device)], dim=1)

 
        if predicted_word in ['.', '?', '!']:
            break

   
    generated_text = tokenizer.decode(tokenized_prompt[0], skip_special_tokens=True)

    return generated_text

@app.route('/generate_text', methods=['POST'])
def generate_text_api():

    data = request.get_json()


    prompt = data['prompt']

    generated_text = generate_text(prompt)

    return jsonify({'generated_text': generated_text})

if __name__ == '__main__':
    app.run(debug=True, port=5000)

